# Aula 19 – Reconhecimento de Padrões: Introdução


## 1. Introdução

Finalmente chegamos no ponto que a “mágica” da Visão Computacional realmente acontece, o reconhecimento de padrões. Entendendo uma imagem como um conjunto de dados (seus pixels), queremos encontrar um padrão nesses dados para reconhecê-lo, isto é, para identificá-lo como algum objeto de interesse. Os algoritmos que fazem esse tratamento são relacionados à Inteligência Artificial e à aprendizagem de máquina.

Todos os processos que estudamos até aqui têm o simples intuito de fazer o tratamento dos dados para facilitar a “vida” dos algoritmos de reconhecimento de padrões, pois quando mais informações mais difícil será para identificar os padrões nos dados. Então os tratamentos dados às imagens, como já discutidos várias vezes, retiram toda a informação que não é importante para que identificação de um objeto de interesse ocorra. E, além disso, como ainda assim os dados podem ser muito grandes mesmo depois de todo o pré-processamento feito, ainda podemos utilizar métodos estatísticos, como PCA, para diminuir sua dimensionalidade. Então os métodos e algoritmos que vamos discutir a partir da aula de hoje assumem que os dados já estão todos tratados e limpos, tornando mais fácil a análise de reconhecimento de padrões.

O primeiro método a ser discutido será o K-NN (K- Nearest Neighbors / K vizinhos mais próximos), um método de classificação simples que compara dados pela distância (geralmente a euclidiana mesmo) entre eles. A ideia é a seguinte: se temos exemplos de dados pertencentes a algumas classes já classificados previamente, um novo dado que queremos classificar assumirá a mesma classe dos k dados de exemplos mais próximos a ele, onde k é um parâmetro dado. Quando k = 1, o novo dado será classificado pelo algoritmo como sendo da mesma classe do exemplo mais próximo a ele. Se k > 1, assumirá a classe que mais aparecer entre os k exemplos mais próximos. Aos dados de exemplo damos o nome de conjunto de treino.

Ou seja, se queremos identificar gatos e cachorros, podemos pegar várias fotos de gatos e várias fotos de cachorro. Fazemos todo o tratamento para que os dados representando as imagens não sejam mais um conjunto gigantesco de pixels, mas sim uma tupla de relativa baixa dimensão de valores, ou seja, vetores relativamente pequenos. A cada um desses vetores, damos a respectiva classe correspondente à imagem original da qual ele foi obtido (gato ou cachorro). Pegamos, então, uma nova imagem que queremos que o algoritmo, por si só, indique se é um cachorro ou um gato. Fazemos exatamente o mesmo pré-processamento da imagem, pegamos o vetor resultante e jogamos para o K-NN, que calculará a distância do vetor da nova imagem para cada um dos outros vetores de exemplo. O K-NN dirá que o novo vetor pertence à classe que mais aparecer entre os K vetores de exemplo mais próximos.

Uma leve alteração que se pode fazer no K-NN é, ao invés de calcular a distância do novo dado aos exemplos, calcular a distância à centroide da classe (média das posições dos exemplos da classe). Esse algoritmo recebe o nome de “Protótipo mais próximo”.

## 2. Leitura Complementar

Livro E – Seção 11.6

Livro 1 – Seções 9.1 e 9.2;

Livro 2 – Capítulos 11 e 12;

Livro 3 - Seções 12.2.1 e 12.2.2

## 3. Exercícios

1. Faça um classificador de imagens que diferencie cachorros e gatos de acordo com o exemplo citado na nota de aula. Defina um tratamento adequado às imagens com redução de dimensionalidade para que o classificador tenha sucesso. Qual a taxa de acerto do seu classificador?

In [8]:
import os
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [9]:
print(len(os.listdir('data/Cat/')))
print(len(os.listdir('data/Dog/')))

12501
12501


In [10]:
try:
    os.mkdir('data/cats-v-dogs')
    os.mkdir('data/cats-v-dogs/training')
    os.mkdir('data/cats-v-dogs/testing')
    os.mkdir('data/cats-v-dogs/training/cats')
    os.mkdir('data/cats-v-dogs/training/dogs')
    os.mkdir('data/cats-v-dogs/testing/cats')
    os.mkdir('data/cats-v-dogs/testing/dogs')
except OSError:
    pass

In [12]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename)

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "data/Cat/"
TRAINING_CATS_DIR = "data/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "data/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "data/Dog/"
TRAINING_DOGS_DIR = "data/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "data/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

TRAINING_CATS_DIR = "data/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "data/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "data/Dog/"
TRAINING_DOGS_DIR = "data/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "data/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)


666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.
666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [13]:
print(len(os.listdir('data/cats-v-dogs/training/cats/')))
print(len(os.listdir('data/cats-v-dogs/training/dogs/')))
print(len(os.listdir('data/cats-v-dogs/testing/cats/')))
print(len(os.listdir('data/cats-v-dogs/testing/dogs/')))

12378
12362
2378
2362


In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [15]:
TRAINING_DIR = "data/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=250,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "data/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=250,
                                                              class_mode='binary',
                                                              target_size=(150, 150))



Found 24739 images belonging to 2 classes.
Found 4739 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator, epochs=15, steps_per_epoch=90,
                    validation_data=validation_generator, validation_steps=6)

Epoch 1/15
55/90 [=================>............] - ETA: 5:57 - loss: 0.9268 - acc: 0.5836

/home/vando/miniconda3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  warnings.warn(
/home/vando/miniconda3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  warnings.warn(
/home/vando/miniconda3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  warnings.warn(
/home/vando/miniconda3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  warnings.warn(
/home/vando/miniconda3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 283
  warnings.warn(
/home/vando/mi

84/90 [===========================>..] - ETA: 1:03 - loss: 0.8344 - acc: 0.5979

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt


acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()